In [2]:
from sentence_transformers import SentenceTransformer

In [5]:
from huggingface_hub import hf_hub_download

In [7]:
hf_hub_download(repo_id="hkunlp/instructor-large", filename="config.json")

'/Users/lichenghu/.cache/huggingface/hub/models--hkunlp--instructor-large/snapshots/54e5ffb8d484de506e59443b07dc819fb15c7233/config.json'

In [9]:
# model_id = "hkunlp/instructor-large"
model = SentenceTransformer("hkunlp/instructor-large")

Using Maximum Sequence Length:  512


In [10]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: T5EncoderModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Dense({'in_features': 1024, 'out_features': 768, 'bias': False, 'activation_function': 'torch.nn.modules.linear.Identity'})
  (3): Normalize()
)

In [11]:
import json
from torch.utils.data import DataLoader
from sentence_transformers import InputExample

In [12]:
BATCH_SIZE = 15

In [13]:
with open("/Users/lichenghu/Desktop/embedding_finetune/train_dataset.json") as jsonfile:
    train_dataset = json.load(jsonfile)

In [14]:
with open("/Users/lichenghu/Desktop/embedding_finetune/val_dataset.json") as jsonfile:
    test_dataset = json.load(jsonfile)

In [15]:
dataset = train_dataset

corpus = dataset['docs']
queries = dataset['queries']
relevant_docs = dataset['relevant_docs']

examples = []
for query_id, query in queries.items():
    node_id = relevant_docs[query_id][0]
    text = corpus[node_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

In [16]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

# Define loss

In [17]:
from sentence_transformers import losses

In [18]:
loss = losses.MultipleNegativesRankingLoss(model)

# Define evaluator

In [19]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

In [20]:
dataset = test_dataset

corpus = dataset['docs']
queries = dataset['queries']
relevant_docs = dataset['relevant_docs']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

In [21]:
EPOCHS = 3

In [ ]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='exp_finetune',
    show_progress_bar=True,
    evaluator=evaluator, 
    evaluation_steps=50,
)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/478 [00:00<?, ?it/s]

Iteration:   0%|          | 0/478 [00:00<?, ?it/s]